In [ ]:
# Install dependencies.
# %%capture

import sys

!{sys.executable} -m pip install -U requests faker

In [ ]:
# Be sure to use your Gretel API key here, which is available from the Profile menu in the Console.

import getpass
import os

gretel_api_key = os.getenv("GRETEL_API_KEY") or getpass.getpass("Your Gretel API Key")

In [ ]:
# Spin up Logstash, Gretel NER, Elasticsearch and Kibana containers.
!GRETEL_API_KEY={gretel_api_key} docker-compose up -d

In [ ]:
# Import dependencies.
from random import random

import requests
from faker import Faker

faker = Faker()


In [ ]:
# Utility to make fake records.
def get_bike_orders(count=0):
    current = 0
    while count == 0 or current < count:
        gender = 'F' if random() < 0.5 else 'M'
        yield {
            "CustomerID": faker.pyint(10000, 99999),
            "Title": None,
            "FirstName": faker.first_name_female() if gender == 'F' else faker.first_name_male(),
            "MiddleName": (faker.first_name_female() if gender == 'F' else faker.first_name_male())
            if random() < 0.5 else None,
            "LastName": faker.last_name(),
            "Suffix": None,
            "AddressLine1": faker.street_address(),
            "AddressLine2": None,
            "City": faker.city(),
            "StateProvinceName": faker.state(),
            "CountryRegionName": "United States",
            "PostalCode": faker.zipcode(),
            "PhoneNumber": faker.phone_number(),
            # EXCLUDE BirthDate, can cause ES to dynamically type body.text as date, not string.
            # "BirthDate": f"{faker.pyint(1930, 2010)}-{faker.pyint(1, 12):02}-{faker.pyint(1, 28):02}",
            "Education": faker.random.choice(["High School", "College Degree", "Graduate Degree"]),
            "Occupation": faker.random.choice(["Disk Jockey", "Burger Jockey", "Thrill Jockey", "Jockey"]),
            "Gender": gender,
            "MaritalStatus": "S" if random() < 0.5 else "M",
            "HomeOwnerFlag": 0 if random() < 0.5 else 1,
            "NumberCarsOwned": faker.pyint(0, 4),
            "NumberChildrenAtHome": faker.pyint(0, 4),
            "TotalChildren": faker.pyint(0, 4),
            "YearlyIncome": faker.pyint(10000, 999999),
            "ExtraText": f"My other address is {faker.ipv4()}." if random() < 0.025 else "Blah blah blah!"
        }
        if count > 0:
            current += 1

In [ ]:
# Logstash listens for new records on this URL.
url = 'http://localhost:8080'

for order in get_bike_orders(count=1000):
    requests.post(url, data=order)

In [ ]:
# Clean up.
!GRETEL_API_KEY={gretel_api_key} docker-compose down